# FFDI CODE: FINAL

All FFDI code in this section was re-run for the later time period (where start_year and end_year were changed)
The outputs of FFDI 1979-1998 and FFDI 2005-2024 were saved as netcdf files
(WARNING: they took about 2.5hrs each to run, on max storage)

## Packages

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import glob
import os
import logging
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

## Preprocessing

In [4]:
# Largely courtesy of Dr Rachael Isphording

In [2]:
# Define bounding box (SWWA)
lat_slice = slice(-36, -32)     
lon_slice = slice(114, 118)

barra11_land_mask = xr.open_dataset('/g/data/ob53/BARRA2/output/reanalysis/AUS-11/BOM/ERA5/historical/hres/BARRA-R2/v1/fx/sftlf/latest/sftlf_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1.nc')
mask = barra11_land_mask['sftlf'].sel(lat=lat_slice, lon=lon_slice)
land_mask = mask > 0

In [3]:
def load_barra_files(path, start_year, end_year, var, 
                     lon_slice=None, lat_slice=None):
    # load land mask
    mask_path = "/g/data/ob53/BARRA2/output/reanalysis/AUS-11/BOM/ERA5/historical/hres/BARRA-R2/v1/fx/sftlf/latest/sftlf_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1.nc"
    land_mask = xr.open_dataset(mask_path)['sftlf']

    if lon_slice and lat_slice:
        land_mask = land_mask.sel(lon=lon_slice, lat=lat_slice)

    land_mask_bool = land_mask >= 50  # boolean mask, 50% of the grid cell or more must be land

    # find files
    all_files = sorted(glob.glob(os.path.join(path, f"{var}_*.nc")))
    
    filtered_files = []
    for f in all_files:
        fname_year = int(os.path.basename(f).split('_')[-1].split('-')[0][:4])
        if start_year <= fname_year <= end_year:
            filtered_files.append(f)
    
    print(f"Found {len(filtered_files)} files for {start_year}-{end_year} in {path}")
    
    # define preprocessing
    def preprocess(ds):
        if lon_slice and lat_slice:
            ds = ds.sel(lon=lon_slice, lat=lat_slice)
        ds[var] = ds[var].where(land_mask_bool) # apply mask
        return ds

    # open files
    ds = xr.open_mfdataset(
        filtered_files,
        combine='by_coords',
        preprocess=preprocess,
        chunks={'time': 365, 'lat': 20, 'lon': 20},  # tweak chunk size for performance
        parallel=True,
    )

    return ds

# Keetch-Byram Drought Index (KBDI)

## KBDI: Precipitation

In [8]:
# LOAD DATA:
# set-up BARRA path and variable
barra_pr_path = "/g/data/ob53/BARRA2/output/reanalysis/AUS-11/BOM/ERA5/historical/hres/BARRA-R2/v1/day/pr/latest"
start_year = 1979
end_year = 1998
variable = "pr"

# load BARRA precipitation data for SWWA
barra_aus11_pr = load_barra_files(barra_pr_path, start_year, end_year, variable,
                                   lat_slice=lat_slice, lon_slice=lon_slice)

Found 240 files for 1979-1998 in /g/data/ob53/BARRA2/output/reanalysis/AUS-11/BOM/ERA5/historical/hres/BARRA-R2/v1/day/pr/latest


In [10]:
precip_mm_day = barra_aus11_pr['pr'] * 86400  # convert to mm/day (BARRA is inkg/m²/s) 
# print(precip_mm_day.head()) # check attributes

In [11]:
# Calculating adjusted rainfall as per Keetch & Bryam (1968)

# define threshold (mm)
threshold = 5.08

def adjusted_rainfall(pr, threshold=5.08):
    # make a copy for output
    adj = pr.copy(deep=True)
    adj[:] = 0  # initialise everything to zero
    
    # convert to numpy for speed (still 3D: time, lat, lon)
    rain = pr.values
    out = np.zeros_like(rain)
    
    # keep cumulative totals for all grid cells at once
    cumulative = np.zeros_like(rain[0])
    active = np.zeros_like(rain[0], dtype=bool)  # whether it's in 'post-threshold' or not
    
    for t in range(rain.shape[0]):
        if not np.all(active):
            cumulative[~active] += rain[t, ~active]
        
        # if threshold is exceeded (and wasn't active before)
        just_activated = (~active) & (cumulative > threshold)
        out[t, just_activated] = cumulative[just_activated] - threshold
        active |= just_activated
        
        # days after threshold is exceeded & rain > 0
        raining_after = active & (rain[t] > 0) & ~just_activated
        out[t, raining_after] = rain[t, raining_after]
        
        # if rain = 0, reset
        reset_cells = (active & (rain[t] == 0))
        active[reset_cells] = False
        cumulative[reset_cells] = 0

    # convert back to DataArray
    adj[:] = out
    adj.name = "adjusted_rainfall"
    adj.attrs["units"] = pr.attrs.get("units", "mm/day")
    adj.attrs["description"] = f"Daily adjusted rainfall after threshold {threshold} mm"
    
    return adj

In [27]:
adjust_rain_test = adjusted_rainfall(precip_mm_day,threshold)
#cell takes a while to run!

In [28]:
adjust_rain_test.compute().max()#.isel(time=0).plot()

# taking largest value for the day & computing it

adjust_rain = adjust_rain_test.compute()

## KBDI: Temperature

In [12]:
# set up BARRA path and variable (years previously defined)
barra_temp_path = "/g/data/ob53/BARRA2/output/reanalysis/AUS-11/BOM/ERA5/historical/hres/BARRA-R2/v1/day/tasmax/latest"
variable_temp = "tasmax" # maximum temp for that day

# load BARRA precipitation data for SWWA
barra_aus11_temp = load_barra_files(barra_temp_path, start_year, end_year, variable_temp,
                                   lat_slice=lat_slice, lon_slice=lon_slice)

Found 240 files for 1979-1998 in /g/data/ob53/BARRA2/output/reanalysis/AUS-11/BOM/ERA5/historical/hres/BARRA-R2/v1/day/tasmax/latest


In [13]:
temp_C = barra_aus11_temp['tasmax'] - 273.15 # convert K to C

# temp_C.isel(time=6190).plot() # sense check

In [32]:
#temp_C.to_netcdf("/g/data/ng72/mf9479/k10_transfer/Thesis/temp_new.nc")

## KBDI: Evapotranspiration (ET) & KBDI calculation

In [26]:
# ET calculation from Keetch & Byram
def calculate_ET(prev_KBDI, temp_C, mean_annual_rainfall):
    numerator = (203.2 - prev_KBDI) * (0.968 * np.exp(0.0875 * temp_C + 1.5552) - 8.30)
    denominator = 1 + 10.88 * np.exp(-0.001736 * mean_annual_rainfall)
    ET = 0.001 * (numerator / denominator)
    return ET

# recursive daily KBDI calculation for a single time series
def kbdi_timeseries(tmax, precip, mean_annual_rainfall, initial_kbdi=50.0):
    # inputs: tmax (daily max temp,°C), precip (daily rainfall, mm), mean_annual_rainfall (mm/year for this grid cell), initial_kbdi (mm) 
    n = len(tmax)
    kbdi = np.zeros(n, dtype=np.float64)
    kbdi[0] = initial_kbdi

    for i in range(1, n):
        prev_KBDI = kbdi[i-1]

        # ET uses yesterday's conditions
        ET = calculate_ET(prev_KBDI, tmax[i-1], mean_annual_rainfall)

        # Net rainfall = max(0, rain - 5)
        net_rain = max(0, precip[i-1] - 5.0)

        # update new KBDI
        new_KBDI = prev_KBDI + ET - net_rain

        # between 0 and 203.2
        new_KBDI = min(max(new_KBDI, 0), 203.2)

        kbdi[i] = new_KBDI

    return kbdi

# make sure time is one chunk (whole series together)
temp_C = temp_C.chunk(dict(time=-1))
adjust_rain = adjust_rain.chunk(dict(time=-1))

def compute_kbdi(ds, initial_kbdi=50.0):
    # tasmax & adjust_rain now in time,lat,lon (not 1D)
    # mean annual rainfall per grid cell (mm/year):
    annual_rain = ds['adjust_rain'].resample(time="YE").sum(dim="time")
    mean_annual_rain = annual_rain.mean(dim="time")  # (lat, lon)

    # run recursive timeseries across all grid cells
    kbdi = xr.apply_ufunc(
        kbdi_timeseries,
        ds["temp_C"],
        ds["adjust_rain"],
        mean_annual_rain,
        kwargs={"initial_kbdi": initial_kbdi},
        input_core_dims=[["time"], ["time"], []],
        output_core_dims=[["time"]],
        vectorize=True,
        dask="parallelized",
        output_dtypes=[np.float64],
    )
    kbdi.name = "kbdi"
    return kbdi

In [27]:
# temp_C : (time, lat, lon) daily max temperature in °C
# adjust_rain : (time, lat, lon) daily rainfall in mm/day

# combine into a dataset
ds = xr.Dataset({
    "temp_C": temp_C,
    "adjust_rain": adjust_rain
})

# run KBDI calculation
kbdi = compute_kbdi(ds, initial_kbdi=50.0)

# attach result
ds["kbdi"] = kbdi

# save to NetCDF (only do this once!)
#ds[["kbdi"]].to_netcdf("KBDI_output.nc")

INFO:flox:Entering _validate_reindex: reindex is None
INFO:flox:Leaving _validate_reindex: method = None, returning None
INFO:flox:_choose_engine: Choosing 'flox'
INFO:flox:_choose_method: method is None
INFO:flox:_choose_method: choosing preferred_method=blockwise
INFO:flox:Entering _validate_reindex: reindex is None
INFO:flox:Leaving _validate_reindex: reindex is False


In [29]:
kbdi.to_netcdf("/g/data/ng72/mf9479/k10_transfer/Thesis/KBDI_old.nc")

## Griffiths' Drought Factor (DF)

In [29]:
# added in, so dont have to reload kbdi 

# KBDI: 2005-2024
kbdi_var_old = xr.open_dataset('/g/data/ng72/mf9479/k10_transfer/Thesis/KBDI_old.nc')
kbdi = kbdi_var_old["kbdi"]

In [19]:
# using Finkele 2006 & Lucas 2010 (eqn from BOM doc, error in griffiths 1999)

In [30]:
def _sig_rain_event(rain, window=20):
    out = np.ones_like(rain, dtype=float)

    for t in range(len(rain)):
        start = max(0, t - window + 1)
        window_rain = rain[start:t+1].copy()
            
        #split the window into events, where each event is n consecutive days of rain, minimum 1 day.
        locs = np.array(np.where(window_rain > 2))[0] #location where days above 2 mm
         # if locs is empty then set so we don't compute P or N and move on
        if np.size(locs) > 0:
            breaks = np.where(np.diff(locs)>1)[0]+1 #splits at the time where the difference in location is more than 1 day
            group = np.split(locs, breaks) #groups the data by the breaks between events
            Ps = [window_rain[g].sum() for g in group] #sums over the groups
            Ns = [len(window_rain) - (g[0] + np.argmax(window_rain[g])) - 1 for g in group]

            # maximise the value for the biggest rain event using the following conditions as outlined in the paper
            tmpx = np.zeros(len(Ps))
            for i in range(len(Ps)):
                # apply piecewise x equation
              N = Ns[i]
              P = Ps[i]
              if (N >= 1) and (P > 2):
                tmpx[i] = (N**1.3) / (N**1.3 + (P - 2))
              elif (N == 0) and (P > 2):
                tmpx[i] = (0.8**1.3) / (0.8**1.3 + (P - 2))
              else:
                tmpx[i] = 1.0
          
            out[t] = np.max(tmpx)
        
        else: out[t] = 1.0
 
    return out

In [31]:
def calculate_sig_rain_event(rain, window=20):
    x = xr.apply_ufunc(
        _sig_rain_event,
        rain,
        input_core_dims=[["time"]],
        output_core_dims=[["time"]],
        vectorize=True,                # loop over lat/lon
        dask="parallelized",           # allow dask execution
        output_dtypes=[float],
        kwargs={"window": window}
    )
    return x

In [32]:
def calculate_x_lim(kbdi):
    return xr.where(kbdi < 20,
                    1 / (1 + 0.1135 * kbdi),
                    75 / (270.525 - 1.267 * kbdi))

def calculate_drought_factor(kbdi_test, rain, window=20):
    x_event = calculate_sig_rain_event(rain, window=window)
    x_lim = calculate_x_lim(kbdi_test)
    x = xr.where(x_event < x_lim, x_event, x_lim)

    x_term = (41 * x**2 + x) / (40 * x**2 + x + 1)
    kbdi_term = 10.5 * (1 - np.exp(-(kbdi_test + 30) / 40.0))
    df = kbdi_term * x_term
    return xr.where(df > 10, 10, df)

In [33]:
precip_mm_day = precip_mm_day.chunk({"time": -1})  # put all time into one chunk
df = calculate_drought_factor(kbdi, precip_mm_day)

In [35]:
df.to_netcdf("/g/data/ng72/mf9479/k10_transfer/Thesis/df_old.nc")

## Windspeed (3pm)

In [2]:
# windspeed = np.sqrt(u**2 + v**2), 
# u is eastward near surface wind (use hourly instantaneous, FFDI usually at 3pm so adjust for UTC. it will be one day out)
# v is northward 
# both are positive, usually m/s (check)

# u = uas (hourly instant), v = vas (hourly instant), select for 3pm 

In [35]:
# THIS IS FOR UAS
barra_uas_path = "/g/data/ob53/BARRA2/output/reanalysis/AUS-11/BOM/ERA5/historical/hres/BARRA-R2/v1/1hr/uas/latest"
variable_uas = "uas"

# Load BARRA precipitation data for subregion
barra_aus11_uas = load_barra_files(barra_uas_path, start_year, end_year, variable_uas,
                                   lat_slice=lat_slice, lon_slice=lon_slice)

Found 240 files for 1979-1998 in /g/data/ob53/BARRA2/output/reanalysis/AUS-11/BOM/ERA5/historical/hres/BARRA-R2/v1/1hr/uas/latest


In [36]:
# THIS IS FOR VAS
barra_vas_path = "/g/data/ob53/BARRA2/output/reanalysis/AUS-11/BOM/ERA5/historical/hres/BARRA-R2/v1/1hr/vas/latest"
variable_vas = "vas"

# Load BARRA precipitation data for subregion
barra_aus11_vas = load_barra_files(barra_vas_path, start_year, end_year, variable_vas,
                                   lat_slice=lat_slice, lon_slice=lon_slice)

Found 240 files for 1979-1998 in /g/data/ob53/BARRA2/output/reanalysis/AUS-11/BOM/ERA5/historical/hres/BARRA-R2/v1/1hr/vas/latest


In [37]:
# Select 07 UTC (which is 3pm Perth)
uas_3pm = barra_aus11_uas.sel(time=barra_aus11_uas.time.dt.hour == 7)
vas_3pm = barra_aus11_vas.sel(time=barra_aus11_vas.time.dt.hour == 7)

# calculate windspeed lazily (otherwise its too computationally massive)
windspeed = np.sqrt(uas_3pm["uas"]**2 + vas_3pm["vas"]**2)

In [28]:
wind_units = xr.open_dataset('/g/data/ob53/BARRA2/output/reanalysis/AUS-11/BOM/ERA5/historical/hres/BARRA-R2/v1/1hr/uas/latest/uas_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_1hr_197902-197902.nc')

In [38]:
windspeed_nc = windspeed * 3.6 (m/s to km/h)

In [39]:
windspeed_nc.to_netcdf("/g/data/ng72/mf9479/k10_transfer/Thesis/windspeed_old.nc")

## Relative Humidity (3pm)

In [14]:
barra_hurs_path = "/g/data/ob53/BARRA2/output/reanalysis/AUS-11/BOM/ERA5/historical/hres/BARRA-R2/v1/1hr/hurs/latest"
variable_hurs= "hurs"

# Load BARRA precipitation data for SWWA
barra_aus11_hurs = load_barra_files(barra_hurs_path, start_year, end_year, variable_hurs,
                                   lat_slice=lat_slice, lon_slice=lon_slice)

Found 240 files for 1979-1998 in /g/data/ob53/BARRA2/output/reanalysis/AUS-11/BOM/ERA5/historical/hres/BARRA-R2/v1/1hr/hurs/latest


In [15]:
# Select 07 UTC (which is 3pm Perth)
hurs_3pm = barra_aus11_hurs["hurs"].sel(time=barra_aus11_hurs.time.dt.hour == 7)

In [7]:
hurs_3pm.to_netcdf("/g/data/ng72/mf9479/k10_transfer/Thesis/rh_new.nc")

# FFDI Calculation

In [22]:
temp_test = temp_C.compute()

# calling temp

In [23]:
# grab variables
temp = temp_test
df = df     # from Griffiths function earlier
rh = hurs_3pm                # relative humidity at 3 pm (%)
wind = windspeed          # wind speed at 10m (km/h)

# strip hours from the 3pm variables so they align with daily datasets (was giving a blank map bc NANs and didnt align by time)
rh = rh.copy()
rh["time"] = rh["time"].dt.floor("D")

wind = wind.copy()
wind["time"] = wind["time"].dt.floor("D")

temp = temp.copy()
temp["time"] = temp["time"].dt.floor("D")

df = df.copy()
df["time"] = df["time"].dt.floor("D")

In [24]:
# avoid log(0) by forcing a minimum drought factor of 1
df_safe = df.clip(min=1)

ffdi = 2 * np.exp(
    -0.45 +
    0.987 * np.log(df_safe) +
    -0.0345 * rh +
    0.0338 * temp +
    0.0234 * (wind * 3.6)   # convert m/s → km/h (assuming your using this code, not previously made netcdf, otherwise conversion is doubles up)
)

# Saving the FFDI files

In [4]:
#ffdi.to_netcdf("/g/data/ng72/mf9479/k10_transfer/Thesis/FFDI.nc") # 2005-2024

In [6]:
#ffdi_old.to_netcdf("/g/data/ng72/mf9479/k10_transfer/Thesis/FFDI_old.nc") # 1979-1998